In [1]:
%load_ext Cython

In [2]:
%%cython
import numpy as np
cimport numpy as np
def c_prefix_sum(int [:] array):
    cdef i = 0
    cdef int [:] b = np.zeros(len(array) + 1, dtype=np.int32)
    for i in range(len(array)):
        b[i+1] = b[i] + array[i]
    return np.asarray(b)

In [3]:
a = np.array([3, 5, 3, 1, 6])
b = np.random.randint(1, 10, 2**16, dtype=np.int32)
c = np.random.randint(1, 10, 2**22, dtype=np.int32)
d = np.random.randint(1, 10, 2**28, dtype=np.int32)

In [4]:
%timeit -r5 -n1 c_prefix_sum(c)

193 ms ± 13.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [5]:
### Cython code works ~10 times faster than Python... Let's check parallel version and compare

In [6]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
import numpy as np
cimport numpy as np
from cython.view cimport array as cvarray
from cython.parallel import prange
from libc.math cimport log2, pow
from cython import boundscheck

@boundscheck(False)
cdef void c_summator(int [:] array, int level, int start, int end, int inc) nogil:
    cdef int sum_index = 0
    for sum_index from start <= sum_index < end by inc:
       array[sum_index] = array[sum_index] + array[sum_index - <int>(pow(2, (level-1)))]

@boundscheck(False)
cdef void c_down_summator(int [:] array, int level, int start, int end, int inc) nogil:
    cdef int sum_index = 0, val = 0
    for sum_index from start <= sum_index < end by inc:
        val = array[sum_index]
        array[sum_index] = array[sum_index] + array[sum_index - <int>pow(2, (level-1))]
        array[sum_index - <int>pow(2, (level-1))] = val


@boundscheck(False)
def c_run(int [:] array, int start, int end, int inc, int max_cores):
    cdef int length = len(array)
    cdef int processing_field, core_number, lbound, rbound, core = 0, level = 0, start_bound, inc_bound
    for level in range(start, end, inc):
        processing_field = int(pow(2, level))
        core_number = int(length / processing_field)
        inc_bound = int(pow(2, level))
        if core_number > max_cores:
            core_number = max_cores
            processing_field = int(length / core_number)
        #print("Level:", level, " Used cores:", core_number)
        for core in prange(core_number, schedule='guided', num_threads=core_number, nogil=True):
            lbound = int(processing_field * core)
            rbound = int(processing_field * (core + 1))
            start_bound = lbound + <int>pow(2, level) - 1
            #print("Level:", level, "   Core: ", core, "   Processing_field: ", processing_field, "   L: ", lbound, "   R: ", rbound)
            if inc == 1:
                c_summator(array, level, start=start_bound, end=rbound, inc=inc_bound)
            else: 
                c_down_summator(array, level, start=start_bound, end=rbound, inc=inc_bound)
    return array


@boundscheck(False)
def c_parallel_prefix_sum(int [:] array, int max_cores=8):
    cdef int length = len(array)
    cdef int depth = int(log2(length))
    cdef int [:] array_view = array
    
    # print("Depth:", depth)
    cdef int [:] b = np.copy(array)
    
    c_run(b, start=1, end=depth + 1, inc=1, max_cores=max_cores)
    b = np.insert(b, length - 1, 0)
    #print("After summator:", b)
    
    b = c_run(b, start=depth, end=0, inc=-1, max_cores=max_cores)
    #print("After down summator:", b)
    return np.asarray(b)

In [7]:
### Let's check speed on small list (2**16 length)
%timeit -r5 -n1 c_prefix_sum(b)
%timeit -r5 -n1 c_parallel_prefix_sum(b, max_cores=1)
%timeit -r5 -n1 c_parallel_prefix_sum(b, max_cores=8)

3.43 ms ± 281 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
2.39 ms ± 634 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
5.54 ms ± 1.14 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [8]:
### When working in more then 1 thread it need some time to start them - so 1 thread works faster

In [9]:
### Let's look at bigger list (2**22 length)
%timeit -r5 -n1 c_prefix_sum(c)
%timeit -r5 -n1 c_parallel_prefix_sum(c, max_cores=1)
%timeit -r5 -n1 c_parallel_prefix_sum(c, max_cores=8)

186 ms ± 8.56 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
138 ms ± 2.71 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
47.5 ms ± 3.43 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [10]:
### Parallel version with 1 thread works slightly faster, and with 8 threads on small array works 2.5 times faster

In [11]:
### Let's check on much longer list (2**28):
%timeit -r2 -n1 c_prefix_sum(d)
%timeit -r2 -n1 c_parallel_prefix_sum(d, max_cores=1)
%timeit -r2 -n1 c_parallel_prefix_sum(d, max_cores=8)

13.8 s ± 229 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
9.71 s ± 169 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
3.98 s ± 6.44 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [13]:
### On much bigger list it still works 2.5 times faster

### Let's check that everything is correct:

In [14]:
np.sum(c_prefix_sum(b))

10746413211

In [15]:
np.sum(c_parallel_prefix_sum(b))

10746413211

In [16]:
c_prefix_sum(b)[0:10]

array([ 0,  5, 11, 15, 20, 21, 28, 35, 36, 45], dtype=int32)

In [17]:
c_parallel_prefix_sum(b)[0:10]

array([ 0,  5, 11, 15, 20, 21, 28, 35, 36, 45], dtype=int32)